In [146]:
aaa.get('data').get('cards')

AttributeError: 'list' object has no attribute 'get'

In [217]:
import requests
from urllib.parse import urlencode
import time
import random
from pyquery import PyQuery as pq
import sys

# 设置代理等（新浪微博的数据是用ajax异步下拉加载的，network->xhr）
host = 'weibo.com'
base_url = 'https://%s/ajax/statuses/repostTimeline?' % host
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'

# 设置请求头
headers = {
    'Host': host,
    'keep': 'close',
    # 'Referer': 'https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&extparam=%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&luicode=10000011&lfid=100103type%3D38%26q%3D%E5%86%85%E5%8D%B7%26t%3D0',
    'User-Agent': user_agent
}
from datetime import datetime
def time_formater(input_time_str):
    input_format = '%a %b %d %H:%M:%S %z %Y'
    output_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(input_time_str, input_format).strftime(output_format)
# 按页数抓取数据
def get_single_page(wid):
    params = {
        'id':wid, # 、、教育内卷、职场内卷、如何看待内卷的社会状态、如何避免婚姻内卷、
    }
    url = base_url + urlencode(params)
    print(url)   
    error_times = 3
    while True:
        response = requests.get(url, headers=headers)  # ,proxies=abstract_ip.get_proxy()
        if response.status_code == 200:
            if len(response.json().get('data')) > 0:
                return response.json()
        time.sleep(3)
        error_times += 1
        # 连续出错次数超过 3
        if error_times > 3:
            return None
count = 0
def parse_page(json_data):
    global count
    items = json_data.get('data')
    for index, item in enumerate(items):
        if item.get('reposts_count')>10:
            data = {
                'wid': item.get('mid'),
                'source': item.get('user').get('screen_name'),
                'target': item.get('retweeted_status').get('user').get('screen_name'),
                'value': item.get('reposts_count'),  # 转发数
                'publish_time': time_formater(item.get('created_at')),
            }
            
            count += 1
            print(f'total count: {count}')
            yield data

def parse_page_twice(json_data,user):
    global count
    items = json_data.get('data')
    for index, item in enumerate(items):
        if item.get('reposts_count')>3:
            data = {
                'wid': item.get('mid'),
                'source': item.get('user').get('screen_name'),
                'target': user,
                'value': item.get('reposts_count'),  # 转发数
                'publish_time': time_formater(item.get('created_at')),
            }
            count += 1
            print(f'total count: {count}')
            yield data
            
def parse_page_third(json_data,user):
    global count
    items = json_data.get('data')
    for index, item in enumerate(items):
        if item.get('reposts_count')>0:
            data = {
                'wid': item.get('mid'),
                'source': item.get('user').get('screen_name'),
                'target': user,
                'value': item.get('reposts_count'),  # 转发数
                'publish_time': time_formater(item.get('created_at')),
            }          
            count += 1
            print(f'total count: {count}')
            yield data
import os, csv

if __name__ == '__main__':
    wid = sys.argv[1]
    
    result_file = f'{wid}.csv'
    if not os.path.exists(result_file):
        with open(result_file, mode='w', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["source","target","value","publish_time"])
    temp_data = []
    temp_wid = []
    temp_wid_twice = []
    temp_user_twice = []
    temp_user = []
    empty_times = 0
    json_data = get_single_page(wid)
    if json_data == None:
        print('json is none')
    for result in parse_page(json_data):  # 需要存入的字段
        temp_data.append(result)
        temp_wid.append(result['wid'])
        temp_user.append(result['source'])
    with open(result_file, mode='a+', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        for d in temp_data:
            writer.writerow(
                [d['wid'], d['source'], d['target'], d['value'], d['publish_time']])
    print(f'\n\n------cur turn write {len(temp_data)} rows to csv------\n\n') 
    time.sleep(random.randint(1, 4))  # 爬取时间间隔
    for i in range(len(temp_wid)):
        temp_data = []
        
        json_data = get_single_page(temp_wid[i])
        if json_data == None:
            print('json is none')
        for result in parse_page_twice(json_data,temp_user[i]):
            temp_data.append(result)
            temp_wid_twice.append(result['wid'])
            temp_user_twice.append(result['source'])
        with open(result_file, mode='a+', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            for d in temp_data:
                writer.writerow(
                    [d['wid'], d['source'], d['target'], d['value'], d['publish_time']])
        print(f'\n\n------cur turn write {len(temp_data)} rows to csv------\n\n')
        time.sleep(random.randint(1, 4))
    for i in range(len(temp_wid_twice)):
        temp_data = []
        json_data = get_single_page(temp_wid_twice[i])
        if json_data == None:
            print('json is none')
        for result in parse_page_third(json_data,temp_user_twice[i]):
            temp_data.append(result)
        with open(result_file, mode='a+', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            for d in temp_data:
                writer.writerow(
                    [d['wid'], d['source'], d['target'], d['value'], d['publish_time']])
        print(f'\n\n------cur turn write {len(temp_data)} rows to csv------\n\n')
        time.sleep(random.randint(1, 4))
            
        
        


https://weibo.com/ajax/statuses/repostTimeline?id=4821006617937520
total count: 1
total count: 2
total count: 3
total count: 4
total count: 5
total count: 6
total count: 7
total count: 8
total count: 9
total count: 10


------cur turn write 10 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821006654637591
total count: 11
total count: 12
total count: 13
total count: 14
total count: 15
total count: 16
total count: 17
total count: 18
total count: 19
total count: 20


------cur turn write 10 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821006654376571
total count: 21
total count: 22
total count: 23
total count: 24
total count: 25
total count: 26
total count: 27
total count: 28
total count: 29
total count: 30


------cur turn write 10 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821006666958466
total count: 31
total count: 32
total count: 33
total count: 34
total count: 35
total count: 36
total count: 37
total count



------cur turn write 0 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821010471456728
total count: 100


------cur turn write 1 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821006865141964
total count: 101


------cur turn write 1 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821013110456550


------cur turn write 0 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821007997602895
total count: 102


------cur turn write 1 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821013155548914


------cur turn write 0 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821012828389842
total count: 103
total count: 104
total count: 105
total count: 106
total count: 107


------cur turn write 5 rows to csv------


https://weibo.com/ajax/statuses/repostTimeline?id=4821013290550974
total count: 108


------cur turn write 1 rows to csv------


https://weibo.com/

In [140]:

host = 'weibo.com'
base_url = 'https://%s/ajax/statuses/repostTimeline?' % host
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/109.0'

# 设置请求头
headers = {
    'Host': host,

    # 'Referer': 'https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&extparam=%23%E5%A6%82%E4%BD%95%E7%9C%8B%E5%BE%85%E5%8F%8D%E5%86%85%E5%8D%B7%E7%83%AD%E6%BD%AE%23&luicode=10000011&lfid=100103type%3D38%26q%3D%E5%86%85%E5%8D%B7%26t%3D0',
    
    'Cookie':'login_sid_t=17456fcbb3fdb83683c780f1b2d6b9aa; cross_origin_proto=SSL; _s_tentry=passport.weibo.com; Apache=1825845321623.7036.1674441601195; SINAGLOBAL=1825845321623.7036.1674441601195; wb_view_log=2048*11522; ULV=1674441602172:1:1:1:1825845321623.7036.1674441601195:; XSRF-TOKEN=F_O9HxGgYROlNTByTF0c-ng_; WBPSESS=MKc6aAHqUx8kbNTT1MYyoYV3qQHwMuA3MDIs_kQ41F2SSF8MoGVl4e_NZXFaJOI_dxyUKLgvq7sYY6-gRvvUecUmbQwKhSiJLVlhQ5J7uPr7pHKoYG01ij0LngQwyveJxbo3NepJAS8p5n0Z-EDPew==; PC_TOKEN=72e5b14d53; SUB=_2A25OyukCDeRhGeBN7lsW9y7PyjmIHXVqNPdKrDV8PUJbkNAKLVGhkW1NRF-7GpvTsnGbXpnIyaHStDaRMo_V-w-J; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhNcd0deeWngaG3B2nCMgDm5NHD95Qce0-4S0M7e02fWs4DqcjVi--Xi-zRiKn7i--fiKysi-8Wi--fi-z7iKysi--fi-z7iKysi--4i-82iKysK7tt;  SSOLoginState=1674484050',
    'User-Agent': user_agent
}

save_per_n_page = 10

In [141]:
response = requests.get(url, headers=headers)  # ,proxies=abstract_ip.get_proxy()
aaa=response.json()

In [139]:
aaa.get('data')[0]

{'visible': {'type': 0, 'list_id': 0},
 'created_at': 'Wed Oct 05 00:31:58 +0800 2022',
 'id': 4821007070924999,
 'idstr': '4821007070924999',
 'mid': '4821007070924999',
 'mblogid': 'M8Oun3SDl',
 'user': {'id': 7060889502,
  'idstr': '7060889502',
  'pc_new': 0,
  'screen_name': '千叶幻灵h',
  'profile_image_url': 'https://tvax2.sinaimg.cn/crop.0.0.1080.1080.50/007HQLUyly8h6wj8jcx18j30u00u077p.jpg?KID=imgbed,tva&Expires=1674497335&ssig=MuXAk8G5Va',
  'profile_url': '/u/7060889502',
  'verified': True,
  'verified_type': 0,
  'domain': '',
  'weihao': '',
  'verified_type_ext': 0,
  'avatar_large': 'https://tvax2.sinaimg.cn/crop.0.0.1080.1080.180/007HQLUyly8h6wj8jcx18j30u00u077p.jpg?KID=imgbed,tva&Expires=1674497335&ssig=idTKFBCFvh',
  'avatar_hd': 'https://tvax2.sinaimg.cn/crop.0.0.1080.1080.1024/007HQLUyly8h6wj8jcx18j30u00u077p.jpg?KID=imgbed,tva&Expires=1674497335&ssig=f1EHrx4HJO',
  'follow_me': False,
  'following': False,
  'mbrank': 6,
  'mbtype': 12,
  'v_plus': 0,
  'planet_video'

In [127]:
from datetime import datetime

def time_formater(input_time_str):
    input_format = '%a %b %d %H:%M:%S %z %Y'
    output_format = '%Y-%m-%d %H:%M:%S'

    return datetime.strptime(input_time_str, input_format).strftime(output_format)
aaa.get('data')[0].get('reposts_count')
aaa.get('data')[0].get('mid')
aaa.get('data')[0].get('reposts_count')
time_formater(aaa.get('data')[0].get('created_at'))

'2022-10-05 00:30:39'

In [116]:
base_url

'https://weibo.com/ajax/statuses/repostTimeline?'

In [135]:
params = {
        'id':aaa.get('data')[0].get('mid'),  # 、、教育内卷、职场内卷、如何看待内卷的社会状态、如何避免婚姻内卷、
}
url = base_url + urlencode(params)
print(url)

https://weibo.com/ajax/statuses/repostTimeline?id=4821006739051447


In [143]:
params = {
    'id':aaa.get('data')[0].get('mid'), # 、、教育内卷、职场内卷、如何看待内卷的社会状态、如何避免婚姻内卷、

}
url = base_url + urlencode(params)
print(url)

https://weibo.com/ajax/statuses/repostTimeline?id=4821007070924999&page=1
